# Integración numérica de la ecuación de onda
Utilizaré distintos métodos (pues el profe hinteó que los algorimos usuales no van a funcionar, pero quisiera ver qué es lo que le pasa) para integrar numéricamente la ecuación de onda adimensionalizada:
$$
\partial_{tt} u(t, x) = \partial_{xx} u(t, x)
$$


## Algoritmo naive:

La idea es usar la clásica discretización de la segunda derivada a ambos lados:

$$
\partial_{zz} f(z) \approx \frac{f(z + h) - 2 f(z) + f(z - h)}{h^2} 
$$

Y con esto, despejar el valor de $u(t + \Delta t, x)$ en función de valores de tiempos anteriores y otras posiciones. Para el primer paso de tiempo simplemente usaré la condición inicial, es decir, supongo velocidad inicial 0 para cada $u(\cdot, x)$. También usaré condiciones de borde fijas.


In [106]:
using LinearAlgebra
using GLMakie

const x_largo = 1.0
const N_x = 101
const Δt = 0.001
const Δx = x_largo/(N_x - 1)
const s = (Δt/Δx)^2
println("s = ", s)
const sMatrix = Tridiagonal(ones(N_x-3) .* s, ones(N_x-2) .* 2 .* (1 - s), ones(N_x-3) .* s)
const bordes = 0.0
const x = range(0, x_largo, length=N_x)

s = 0.010000000000000002


0.0:0.01:1.0

In [107]:
function paso_temporal!(u, u_anterior)
    buffer = copy(u)
    u[2:end-1] .= sMatrix * u[2:end-1] .- u_anterior[2:end-1]
    u[1] = bordes
    u[end] = bordes # En verdad esto no es necesario.
    u[2] += s * bordes 
    u[end-1] += s * bordes
    u_anterior .= buffer
end

function actualizar_animacion!(i)
    lines[1][] = Point2f[[x[i], u[i]] for i in 1:N_x] # Detesto tener que hacer esto.
    paso_temporal!(u, u_anterior)
    return
end

actualizar_animacion! (generic function with 1 method)

In [108]:
# Animación:

u = sin.(2π .* x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1])
lines = lines!(ax, x, u)

nframes = 480
framerate = 60

record(actualizar_animacion!, fig, "modo_normal.mp4", 1:nframes; framerate=framerate)


"modo_normal.mp4"

Ahora otra condición inicial

In [109]:
function u_func(x)
    if x < 1/3 || x > 2/3
        return 0.0
    elseif x < 1/2
        return x - 1/3
    else
        return 2/3 - x
    end
end


u = u_func.(x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1])
lines = lines!(ax, x, u)

nframes = 480
framerate = 60

record(actualizar_animacion!, fig, "triángulo.mp4", 1:nframes; framerate=framerate)


"triángulo.mp4"

Vemos que hay unas oscilaciones extrañas ahí metidas. Creo que debería hacer un análisis de estabilidad para ver qué está pasando y por qué.